<a href="https://colab.research.google.com/github/Shaifali07/LLM_Application_Practice/blob/main/Output_parsers_langcain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install langchain_core
!pip install langchain_groq
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.7 MB/s eta 0:00:00


In [2]:
from my_key import grq_key
import os
os.environ["GROQ_API_KEY"]=grq_key


In [53]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    model_name="gemma2-9b-it",
    temperature=0
)

In [4]:
from langchain import PromptTemplate

OUTPUT PARSERS

StrOutputParser

In [54]:
from langchain_core.output_parsers import StrOutputParser
my_prompt=PromptTemplate(
    input_variables=["word"],
    template="Give a definion of the {word} in one line"
)
chain_without_parser=my_prompt|llm
s=chain_without_parser.invoke("atmosphere")
print(s)
print("------------------------------------------------------------------------------------------------")
chain_with_parser=my_prompt|llm|StrOutputParser()
chain_with_parser.invoke("atmosphere")

content='The atmosphere is the layer of gases surrounding a planet, held in place by gravity. \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 19, 'total_tokens': 40, 'completion_time': 0.038181818, 'prompt_time': 7.846e-05, 'queue_time': 0.021910949000000002, 'total_time': 0.038260278}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-6f37e554-be25-4f89-8e20-bef914926fef-0' usage_metadata={'input_tokens': 19, 'output_tokens': 21, 'total_tokens': 40}
------------------------------------------------------------------------------------------------


'The atmosphere is the layer of gases surrounding a planet, held in place by gravity. \n'

StructuredOutputParser

In [56]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
response_schema=[ResponseSchema(name="definition", description="definition of the word in one line"),ResponseSchema(name="synonym",description="synonym of the given word")]
output_parser=StructuredOutputParser.from_response_schemas(response_schema)
format_instructions=output_parser.get_format_instructions()
print(format_instructions)
my_prompt_for_structured_parser=PromptTemplate(
    input_variables=['Word'],
    template="Give one line definiton of the word and its synonyms for the given : {Word}\n {format_instructions}",
    partial_variables={'format_instructions':format_instructions}
)
chain_with_structured_parser=my_prompt_for_structured_parser|llm|output_parser
chain_with_structured_parser.invoke("atmosphere")

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"definition": string  // definition of the word in one line
	"synonym": string  // synonym of the given word
}
```


{'definition': 'The layer of gases surrounding a planet.', 'synonym': 'air'}

Comma Seperated Parser

In [62]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser=CommaSeparatedListOutputParser()
format_instructions=output_parser.get_format_instructions()
print(format_instructions)
prompt_for_comma_seperated_parser=PromptTemplate(
    input_variables=["word"],
    template="give synonyms of {word}. \n {format_instructions}",
    partial_variables={"format_instructions": format_instructions}
)
chain_with_comma_seperated_parser=prompt_for_comma_seperated_parser|llm|output_parser
chain_with_comma_seperated_parser.invoke({"word":"atmosphere"})

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


['ambiance',
 'mood',
 'climate',
 'air',
 'environment',
 'temper',
 'spirit',
 'tone',
 'aura',
 'feeling',
 'vibe  ']

DateTimeOutputParser

In [67]:
from langchain.output_parsers import DatetimeOutputParser
output_parser=DatetimeOutputParser()
format_instructions=output_parser.get_format_instructions()
print(format_instructions)
prompt_for_datetime_parser=PromptTemplate(
    input_variables=["Question"],
    template="give answer of question {Question}. \n {format_instructions}",
    partial_variables={"format_instructions": format_instructions}
)
chain_with_datetime_parser=prompt_for_datetime_parser|llm|output_parser
s=chain_with_datetime_parser.invoke({"Question":"When india won its first worldcup?"})
print(s)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 181-08-28T02:42:18.761391Z, 1512-10-18T19:56:37.234879Z, 1861-09-05T18:15:18.860992Z

Return ONLY this string, no other words!
1983-06-25 14:00:00
